## Introduction

This notebook tries to explore the dual concepts of data and recursion, and how they can be applied to common programming tasks.

**Data** represents finite values. A given data $x$ of type $T$ is made by a finite application of _constructors_ of $T$. For eg, any finite stack is made by creating a new stack and then pushing a finite amount of values.

**Codata** (usually) represents infinite values. A given codata is defined by specifying a set of _destructor_ functions. A destructor is a function able to extract some part of the infinite structure (like `head` and `tail`).

## Recursion and Corecursion

> In computer science, corecursion is a type of operation that is dual to recursion. Whereas recursion works analytically, starting on data further from a base case and breaking it down into smaller data and repeating until one reaches a base case, corecursion works synthetically, starting from a base case and building it up, iteratively producing data further removed from a base case. [wikipedia](https://en.wikipedia.org/wiki/Corecursion)

**Recursion** can be used to define functions that map values from data $x$ by invoking itself over the parts of $x$. These functions will eventually stop recursing when reaching the base cases.

**Corecursion** defines functions that map values from _codata_ by applying destructors to the results.

Consider a recursive function to add one to every element of a list of numbers:

In [ ]:
def add(ns):
  if ns==[]:
    return []
  n, *ns = ns
  return [n+1] + add(ns)

assert add([1,2,3]) == [2,3,4]

> Corecursion is often used in conjunction with lazy evaluation, to produce only a finite subset of a potentially infinite structure (rather than trying to produce an entire infinite structure at once)

In Python we use generators to achieve lazy evaluation:

In [ ]:
# codata, list of all naturals
def nats(i=1):
  yield i
  yield from nats(i+1)

Function `head` shows the first $n$ elements of a given list,

In [ ]:
def head(xs, n):
  """ generates the first n items """
  for i,x in enumerate(xs):
    if i==n:
      break
    yield x

print(*head(nats(), 30))

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30


Let's now make a corecursive function to add one to all naturals:

In [ ]:
def coadd(ns):
  yield next(ns)+1
  yield from coadd(ns)

print(*head(coadd(nats()), 30))

2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31


Another codata eg, an infinite list of ones:

In [ ]:
# In Haskell: ones = 1 : ones
def ones():
  yield 1
  yield from ones()

In [ ]:
print(*head(ones(), 10))

1 1 1 1 1 1 1 1 1 1


Corecursion does not need always to create infinite codata. The next corecursive function produces a finite list when $a\leq b$,

In [ ]:
def count_upto(a, b):
  if a != b+1:
    yield a
    yield from count_upto(a+1,b)

In [ ]:
print(*head(count_upto(2, 9), 30))
print(*head(count_upto(9, 2), 30))

2 3 4 5 6 7 8 9
9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38


Corecursion creates a _stream_ of values, starting from the bases of recursion to more complex subproblems.

In [ ]:
def facts(i=0, fac=1):
  yield fac
  yield from facts(i+1, fac*(i+1))

In [ ]:
print(*head(facts(), 15))

1 1 2 6 24 120 720 5040 40320 362880 3628800 39916800 479001600 6227020800 87178291200


To make a corecursive generator, we can do without recursion,

In [ ]:
# alternatives to previous implementations
def nats():
  i = 1
  while True:
    yield i
    i += 1

def coadd(ns):
  for n in ns:
    yield n+1    

Let's check some more examples.

Returns the odd-indexed elements:

In [ ]:
def odds(xs):
  while True:
    yield next(xs)
    next(xs)

print(*head(odds(facts()), 10))

1 2 24 720 40320 3628800 479001600 87178291200 20922789888000 6402373705728000


Python iterable functions work well in this context:

In [ ]:
# remove all factorials having one or more digits 3
not3 = lambda n: '3' not in str(n)

print(*head(filter(not3, facts()), 10))

1 1 2 6 24 120 720 5040 479001600 6227020800


A stream of all factorials,

In [ ]:
def facts():
  i, fac = 0, 1
  while True:
    yield fac
    i, fac = i+1, fac*(i+1)

The Fibonacci sequence,

In [ ]:
def fibs():
  a, b = 0, 1
  while True:
    yield a
    a, b = b, a + b

In [ ]:
print(*head(fibs(), 25))

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368


A corecursive generator for $\mathbb{Q}$,

In [ ]:
def rationals():
  yield (1,1)
  a = [1,1]
  k = 1
  while True:
    if k % 2 == 0:
      a.append( a[k//2] )
    else:
      kHalf = k//2
      a.append (a[kHalf]+a[kHalf+1] )
    yield (a[k], a[k+1])
    k = k+1

In [ ]:
print(*head(rationals(), 15))

(1, 1) (1, 2) (2, 1) (1, 3) (3, 2) (2, 3) (3, 1) (1, 4) (4, 3) (3, 5) (5, 2) (2, 5) (5, 3) (3, 4) (4, 1)


The corecursive version of the sieve of Eratosthenes:

In [ ]:
from itertools import count

def sieve(ns):
  n = next(ns)
  yield n
  yield from sieve( i for i in ns if i%n!=0 )

def primes():
  yield from sieve(count(start=2))

In [ ]:
print(*head(primes(), 50))

2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97 101 103 107 109 113 127 131 137 139 149 151 157 163 167 173 179 181 191 193 197 199 211 223 227 229


Corecursivity can be applied to other codatatypes. The next generator traverses an infinite binary tree using breath first:

In [ ]:
from queue import deque

class Tree:
  def __init__(self, val, left, right):
    self.val   = val
    self.left  = left
    self.right = right

def bf(tree):
  nodes = deque([tree])
  while nodes:
    t = nodes.popleft() 
    if t is not None:
      yield t.val
      nodes.append(t.left)
      nodes.append(t.right)

In [ ]:
t1 = Tree(1, None, None)
t2 = Tree(2, None, None)
t3 = Tree(3, t1, t2)
t1.left  = t3 # make an infinite tree    
t1.right = t3

print(*head(bf(t3), 50))

3 1 2 3 3 1 2 1 2 3 3 3 3 1 2 1 2 1 2 1 2 3 3 3 3 3 3 3 3 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 3 3 3 3 3


Streams can also be defined corecursively.

Consider the following corecursive Haskell definition for the Fibonacci sequence,

    fibs = 0 : 1 : zipWith (+) fibs (tail fibs)

Let's translate that to Python,    

In [ ]:
from itertools import tee, islice, chain
from operator import add

def fibs():
  def output():
    for i in chain((0,1), map(add, fib, tail)):
      yield i

  stream, fib, tail = tee(output(), 3)
  tail = islice(tail, 1, None)
  return stream

In [ ]:
print(*head(fibs(), 25))

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368


### Unfolds

A **fold** is a recursive function that analyses a data structure and recombines their elements (usually by summarizing them).

Python provides a folding function denoted `reduce`,

In [ ]:
from functools import reduce

xs = [1,2,3,4]
reduce(lambda acc,x: 2*x+acc, xs, 0)

20

An **unfold** is a corecursive function that generates a sequence by repeated application of a given function to its previous result.

A famous unfold is `iterate`:

In [ ]:
def iterate(f, x):
  yield x
  yield from iterate(f, f(x))

# alternative 
def iterate(f, x):
  while True:
    yield x
    x = f(x)

Making the powers of 2 using `iterate`:

In [ ]:
print(*head(iterate(lambda x:2*x, 1), 20))

1 2 4 8 16 32 64 128 256 512 1024 2048 4096 8192 16384 32768 65536 131072 262144 524288


Let's make a more general unfold factory:

In [ ]:
def unfold(step, end, state):
  while True:
    if end(state):
      return
    value, state = step(state)
    yield value

forever = lambda _: False    

Some use examples:

In [ ]:
powers2 = unfold(lambda st: (st,2*st), forever, 1)
print(*head(powers2, 20))

1 2 4 8 16 32 64 128 256 512 1024 2048 4096 8192 16384 32768 65536 131072 262144 524288


Defining `iterate` via `unfold`,

In [ ]:
iterate = lambda f, n: unfold(lambda st: (st,f(st)), forever, n)

powers3 = iterate(lambda x:3*x, 1)
print(*head(powers3, 20))

1 3 9 27 81 243 729 2187 6561 19683 59049 177147 531441 1594323 4782969 14348907 43046721 129140163 387420489 1162261467


This next function is a zip for two indexed structures:

In [ ]:
end  = lambda ps: not ps[0] or not ps[1]
step = lambda ps: ((ps[0][0],ps[1][0]), (ps[0][1:],ps[1][1:]))
zip2 = lambda xs, ys: unfold(step, end, (xs,ys))

print(*zip2('abcde',[1,2,3,4,5,6]))

('a', 1) ('b', 2) ('c', 3) ('d', 4) ('e', 5)


## Trees as codata

A board game, like Tic-Tac-Toe, can be represented as a tree, where nodes are valid positions and, given each node, its children represent all valid positions achievable by playing one of the available legal moves.

Here's a snapshot of the first nodes of Tic-Tac-Toe game tree,

<center><img src='https://raw.githubusercontent.com/jpneto/Prog.I/master/imgs/tree_tictactoe_full.png' width=700px></center>

Consider a tree as a node with a state (the necessary information to define a valid game position), and a list of nodes representing the next valid positions (its children),

In [5]:
class Tree:
  def __init__(self, state, children):
    self.state    = state
    self.children = children # list of Trees

  def __iter__(self):
    yield from tree_gen(self)

  def size(self, t):
    if t is None:
      return 0
    return 1 + sum(child.size() for child in t.children)
  
  def __repr__(self):
    return self._make_repr(0)

  def _make_repr(self, level=0):
    ret = ["   "*level + repr(self.state)]
    for child in self.children:
      ret.append(child._make_repr(level+1))
    return '\n'.join(ret)

The class is iterable, using breadth-first,

In [2]:
from queue import Queue

def tree_gen(t): 
  q = Queue() # breadth-first
  q.put(t)
  while not q.empty():
    t = q.get()
    yield t.state    
    for child in t.children:
      q.put(child)

To build the game tree we could implement a recursive funciton that would go thru all computed positions to compute the next level, and on...

Another way is to define the game tree in a corecursive way.

Assume we have function `next_states :: state -> [state]` that, given a state, returns all next valid states. Given an initial state (in Tic-Tac-Toe, the empty board), a game tree is defined by the next corecursive function,

In [6]:
def iterate_tree(next_states):
  return lambda state: Tree(state, 
                            map(iterate_tree(next_states), next_states(state)))

Notice all the game tree is lazely defined. The game tree might be huge, or even infinite (for some other games), but its lazy nature allows us to compute only parts, if we wish so!

Usually we only require to compute a finite depth of the game tree. That's the task of `prune`,

In [7]:
def prune(level):
  def prunner(tree):
    if level==0:
      return Tree(tree.state, [])
    # list computes the concrete tree, instead of each children being an iterable
    return Tree(tree.state, list(map(prune(level-1), tree.children)))
  return prunner

### Tic-Tac-Toe

Let's code the game rules of Tic-Tac-Toe,

In [8]:
class TTT:
  """ a tictactoe state """
  def __init__(self, board, next_player):
    self.board       = board
    self.value       = 0
    self.next_player = next_player

  def __repr__(self):
    return (''.join(self.board[ :3]) + '/' +
            ''.join(self.board[3:6]) + '/' +
            ''.join(self.board[6: ]) + 
            ' [' + self.next_player + ']' +
            ' ' + str(self.value))

The next predicate checks if the state is an endgame.

In [9]:
def has_win(board, player):
  for i in range(3):
    if board[3*i:3*i+3] == [player]*3:            # check row
      return True
    if board[i::3]      == [player]*3:            # check column
      return True
  if board[0] == board[4] == board[8] == player:  # check main diagonal
    return True
  if board[2] == board[4] == board[6] == player:  # check anti diagonal
    return True    
  return False

assert has_win(list('XXX......'), 'X') # rows
assert has_win(list('...OOO...'), 'O')
assert has_win(list('......XXX'), 'X')
assert has_win(list('X..X..X..'), 'X') # columns
assert has_win(list('.X..X..X.'), 'X')
assert has_win(list('..O..O..O'), 'O')
assert has_win(list('X...X...X'), 'X') # diagonals
assert has_win(list('..O.O.O..'), 'O')

assert not has_win(list('XXOOOXXXO'), 'X')

The `next_states` for Tic-Tac-Toe:

In [11]:
def next_ttt_states(ttt_game):
  board, player = ttt_game.board, ttt_game.next_player

  new_states = []
  next_player = 'X' if player == 'O' else 'O'
  if not has_win(board, next_player): # there's only states if game has not ended
    for i in range(9):
      if board[i] == '.':
        new_board = board[:]
        new_board[i] = player
        new_states.append(TTT(new_board, next_player))
  return new_states

Let's define the game tree (define, not compute it yet),

In [12]:
init_state = TTT(list('.'*9), 'X') # empty board
game_tree = iterate_tree(next_ttt_states)(init_state)

Let's check its first level, i.e., the valid moves for first player:

In [13]:
prune(1)(game_tree)

.../.../... [X] 0
   X../.../... [O] 0
   .X./.../... [O] 0
   ..X/.../... [O] 0
   .../X../... [O] 0
   .../.X./... [O] 0
   .../..X/... [O] 0
   .../.../X.. [O] 0
   .../.../.X. [O] 0
   .../.../..X [O] 0

Another example, taking a position with some stones, check the ramaining game tree:

In [17]:
state = TTT(list('XX.O.O.XO'), 'X')
game_tree = iterate_tree(next_ttt_states)(state)

end_game = prune(3)(game_tree)
end_game

XX./O.O/.XO [X] 0
   XXX/O.O/.XO [O] 0
   XX./OXO/.XO [O] 0
   XX./O.O/XXO [O] 0
      XXO/O.O/XXO [X] 0
      XX./OOO/XXO [X] 0

Right now, the value attribute for each state is still zero. 

Function `eval_board` assigns values to end positions, 

In [15]:
def eval_board(board):
  if has_win(board, 'X'):
    return 1
  if has_win(board, 'O'):
    return -1
  return 0

> it would be possible to create elaborate heuristics to assign numbers between `[-1,1]` to help a computer player to navigate the game tree, but that's not our goal here.

The next function initializes the (possibly pruned) game tree with its winning positions,

In [16]:
def eval_tree(f, tree):
  tree.state.value = eval_board(tree.state.board)
  for child in tree.children:
    eval_tree(f, child)

In [19]:
eval_tree(eval_board, end_game)
end_game

XX./O.O/.XO [X] 0
   XXX/O.O/.XO [O] 1
   XX./OXO/.XO [O] 1
   XX./O.O/XXO [O] 0
      XXO/O.O/XXO [X] -1
      XX./OOO/XXO [X] -1

We can now apply min-max to the game tree, to decide, for every upper node, if there's a winning strategy,

In [20]:
def max_tree_value(tree):
  if tree.children:
    tree.state.value = max(map(min_tree_value, tree.children))
  return tree.state.value

def min_tree_value(tree):
  if tree.children:
    tree.state.value = min(map(max_tree_value, tree.children))
  return tree.state.value

In [21]:
max_tree_value(end_game)
end_game # if node is 1/-1, there's a winnable position for X/O

XX./O.O/.XO [X] 1
   XXX/O.O/.XO [O] 1
   XX./OXO/.XO [O] 1
   XX./O.O/XXO [O] -1
      XXO/O.O/XXO [X] -1
      XX./OOO/XXO [X] -1

A larger game tree example,

In [22]:
state = TTT(list('X..O..X.O'), 'X')                # game: X..
game_tree = iterate_tree(next_ttt_states)(state)   #       O..
game2 = prune(5)(game_tree)                        #       X.O

eval_tree(eval_board, game2)
max_tree_value(game2)

game2 # it's possible to win if X plays at upper-right corner

X../O../X.O [X] 1
   XX./O../X.O [O] 0
      XXO/O../X.O [X] 0
         XXO/OX./X.O [O] -1
            XXO/OXO/X.O [X] -1
            XXO/OX./XOO [X] 0
               XXO/OXX/XOO [O] 0
         XXO/O.X/X.O [O] 0
            XXO/OOX/X.O [X] 0
               XXO/OOX/XXO [O] 0
            XXO/O.X/XOO [X] 0
               XXO/OXX/XOO [O] 0
         XXO/O../XXO [O] -1
            XXO/OO./XXO [X] 0
               XXO/OOX/XXO [O] 0
            XXO/O.O/XXO [X] -1
      XX./OO./X.O [X] 1
         XXX/OO./X.O [O] 1
         XX./OOX/X.O [O] 0
            XXO/OOX/X.O [X] 0
               XXO/OOX/XXO [O] 0
            XX./OOX/XOO [X] 1
               XXX/OOX/XOO [O] 1
         XX./OO./XXO [O] -1
            XXO/OO./XXO [X] 0
               XXO/OOX/XXO [O] 0
            XX./OOO/XXO [X] -1
      XX./O.O/X.O [X] 1
         XXX/O.O/X.O [O] 1
         XX./OXO/X.O [O] -1
            XXO/OXO/X.O [X] -1
            XX./OXO/XOO [X] 1
               XXX/OXO/XOO [O] 1
         XX./O.O/XXO [O] -1
            X

## References

+ Turner - [Total Functional Programming](http://sblp2004.ic.uff.br/papers/turner.pdf) (2004)

+ Mike Gordon - [Corecursion and Coinduction](https://www.semanticscholar.org/paper/Corecursion-and-coinduction-%3A-what-they-are-and-how-Gordon/41fb876f6b35971173ef1808472350b51cf3afd1) (2007)

+ John Hughes - [Why Functional Programming Matters](https://www.cs.kent.ac.uk/people/staff/dat/miranda/whyfp90.pdf) (1990)

+ Benjamin Jones - [Alpha-Beta Pruning](https://web.archive.org/web/20180819083507/http://bfj7.com/blog/alpha-beta-pruning.html) (2013)

+ Wikipedia, [Corecursion](https://en.wikipedia.org/wiki/Corecursion)